In [1]:
import sys

sys.path.append("../")

In [2]:
import io
import json
import os

import boto3
from dotenv import load_dotenv
import pandas as pd

from lambda_pivot_health_variables.pivot_health_variables import pivot_variables


load_dotenv()

True

# Try Python functions locally

In [3]:
selected_variable_codes_file = "./food_environment_atlas_selected_variables.txt"
selected_variable_codes = pd.read_csv(selected_variable_codes_file, header=None).squeeze("columns")

query_string_parameters = dict(
    bucket="udacity-nanodegree-capstone-project",
    input_file="StateAndCountyData.csv",
    output_file="StateAndCountyDataClean.csv",
    selected_variable_codes=selected_variable_codes
)

pivot_variables(dict(queryStringParameters=query_string_parameters), None)

{'statusCode': 200, 'body': 'Data was processed correctly'}

In [4]:
from lambda_pivot_health_variables.pivot_health_variables import _aws_s3_csv_to_stream


pd.read_csv(_aws_s3_csv_to_stream("udacity-nanodegree-capstone-project", "StateAndCountyDataClean.csv"))

,FIPS,State,County,PCT_LACCESS_POP15,PCT_LACCESS_LOWI15,PCT_LACCESS_SNAP15,PCT_LACCESS_CHILD15,PCT_LACCESS_SENIORS15,PCT_LACCESS_WHITE15,PCT_LACCESS_BLACK15,...,PCT_NHPI10,PCT_65OLDER10,PCT_18YOUNGER10,MEDHHINC15,POVRATE15,PERPOV10,CHILDPOVRATE15,PERCHLDPOV10,METRO13,POPLOSS10
0,1,AL,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,DE,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,DE,Kent County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,DE,Kent,15.755920,4.461135,2.620211,3.944441,2.350658,11.177089,3.268409,...,0.045592,13.543220,24.877703,55678.0,14.2,0.0,21.3,0.0,1.0,0.0
4,10003,DE,New Castle County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6331,9011,CT,New London,36.134181,7.148509,3.824637,7.322355,5.465421,29.891950,2.075558,...,0.058018,14.228896,21.747095,65602.0,11.1,0.0,17.4,0.0,1.0,0.0
6332,9013,CT,Tolland County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6333,9013,CT,Tolland,34.931152,8.011685,2.849583,6.988324,4.189072,31.098412,1.319530,...,0.018993,11.932596,20.226470,81616.0,7.1,0.0,7.2,0.0,1.0,0.0
6334,9015,CT,Windham County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Try Docker image locally

In [8]:
import requests


response = requests.get(
    f"http://localhost:9000/2015-03-31/functions/function/invocations",
    json={"queryStringParameters": {
        "bucket": "udacity-nanodegree-capstone-project",
        "input_file": "StateAndCountyData.csv",
        "output_file": "StateAndCountyDataCleanNotebook.csv",
        "selected_variable_codes": ["PCT_LACCESS_POP15"]
    }}
)

In [9]:
response.content

b'{"statusCode": 200, "body": "Data was processed correctly"}'

# Try AWS API Gateway

In [14]:
import requests
import subprocess


process = subprocess.Popen(["terraform", "output", "-raw", "api_gateway_base_url"], cwd="../terraform/etl_pipeline_infrastructure", stdout=subprocess.PIPE)
terraform_output = process.stdout.read().decode('utf-8')

response = requests.get(
    f"{terraform_output}/hello",
    params={
        "bucket": "udacity-nanodegree-capstone-project",
        "input_file": "StateAndCountyData.csv",
        "output_file": "StateAndCountyDataCleanNotebookAPI.csv",
        "selected_variable_codes": ["PCT_LACCESS_POP15", "PCT_LACCESS_SNAP15"]
    }
)

response.content

In [8]:
requests.get(
    f"http://localhost:8000/hello",
    params={
        "bucket": "udacity-nanodegree-capstone-project",
        "input_file": "StateAndCountyData.csv",
        "output_file": "StateAndCountyDataCleanNotebookAPI.csv",
        "selected_variable_codes": ["PCT_LACCESS_POP15", "PCT_LACCESS_SNAP15"]
    }
)

<Response [404]>